# MLP Backpropagation

## Forward Pass

Let’s assume a neural network with two layers. For simplicity, let:
    1. $\mathbf{x}$ \in $\mathbb{R}^n$ be the input vector.
    2. $\mathbf{W}_1$ \in $\mathbb{R}^{m \times n}$ and $\mathbf{b}_1$ \in $\mathbb{R}^m$ be the weight matrix and bias vector for the first layer.
    3. $\mathbf{W}_2$ \in $\mathbb{R}^{k \times m}$ and $\mathbf{b}_2 \in \mathbb{R}^k$ be the weight matrix and bias vector for the second (output) layer.
    4. $\sigma(\cdot)$ is an activation function (such as sigmoid or ReLU).

### Step 1: First Layer Computations
1. Compute the weighted sum of inputs for the first layer:
    $$
   \mathbf{z}_1 = \mathbf{W}_1 \mathbf{x} + \mathbf{b}_1
   $$
2. Apply the activation function to get the activations of the first layer:
    $$
   \mathbf{a}_1 = \sigma(\mathbf{z}_1)
    $$

###  2: Second Layer Computations (Output Layer)
1. Compute the weighted sum for the second layer:
    $$
   \mathbf{z}_2 = \mathbf{W}_2 \mathbf{a}_1 + \mathbf{b}_2
   $$

2. Apply the activation function (or softmax if it’s for classification) to get the final output:
   $$
   \mathbf{y} = \sigma(\mathbf{z}_2)
   $$

Here, $\mathbf{y}$ is the network’s prediction for a given input $\mathbf{x}$.

---

## Backpropagation Using Chain Rule

Let’s assume a loss function $L(\mathbf{y}, \mathbf{t})$, where $\mathbf{t}$ is the target output vector. We’ll backpropagate to update $\mathbf{W}_2$, $\mathbf{b}_2$, $\mathbf{W}_1$, and $\mathbf{b}_1$ using gradients obtained via the chain rule.

### Step 1: Compute the Derivative of the Loss with respect to $\mathbf{z}_2$

1. The loss gradient with respect to the output $\mathbf{y}$ is:
    $$
   \frac{\partial L}{\partial \mathbf{y}} = \nabla_{\mathbf{y}} L
   $$
2. Using the chain rule, compute the derivative of L with respect to $\mathbf{z}_2$:
    $$
   \frac{\partial L}{\partial \mathbf{z}_2} = \frac{\partial L}{\partial \mathbf{y}} \circ \sigma'(\mathbf{z}_2)
   $$
   where $\circ$ denotes element-wise multiplication and $\sigma'(\mathbf{z}_2)$ is the derivative of the activation function applied element-wise to $\mathbf{z}_2$.

### Step 2: Derivatives with respect to $\mathbf{W}_2$ and $\mathbf{b}_2$

1. Using the chain rule, compute the gradient of the loss with respect to $\mathbf{W}_2$:
    $$
   \frac{\partial L}{\partial \mathbf{W}_2} = \frac{\partial L}{\partial \mathbf{z}_2} \cdot \mathbf{a}_1^\top
   $$
2. Similarly, compute the gradient with respect to $\mathbf{b}_2$:
    $$  
    \frac{\partial L}{\partial \mathbf{b}_2} = \frac{\partial L}{\partial \mathbf{z}_2}
    $$

#### Step 3: Backpropagate to the First Layer

Now, propagate the error back to the first layer to compute the gradients with respect to $\mathbf{W}_1$ and $\mathbf{b}_1$.

1. Compute the error at the first layer by backpropagating $\frac{\partial L}{\partial \mathbf{z}_2}$ through $\mathbf{W}_2$:
    $$  
   \frac{\partial L}{\partial \mathbf{z}_1} = (\mathbf{W}_2^\top \cdot \frac{\partial L}{\partial \mathbf{z}_2}) \circ \sigma'(\mathbf{z}_1)
   $$
2. Compute the gradient of the loss with respect to $\mathbf{W}_1$:
    $$
   \frac{\partial L}{\partial \mathbf{W}_1} = \frac{\partial L}{\partial \mathbf{z}_1} \cdot \mathbf{x}^\top
   $$
3. Compute the gradient with respect to $\mathbf{b}_1$:
    $$
   \frac{\partial L}{\partial \mathbf{b}_1} = \frac{\partial L}{\partial \mathbf{z}_1}
   $$

---

### Final Backpropagation Updates

With the gradients computed, update each parameter using a learning rate \eta:

1. Update weights and biases for the second layer:
    $$
   \mathbf{W}_2 := \mathbf{W}_2 - \eta \frac{\partial L}{\partial \mathbf{W}_2}
   $$
   $$
   \mathbf{b}_2 := \mathbf{b}_2 - \eta \frac{\partial L}{\partial \mathbf{b}_2}
    $$

2. Update weights and biases for the first layer:
    $$
   \mathbf{W}_1 := \mathbf{W}_1 - \eta \frac{\partial L}{\partial \mathbf{W}_1}
   $$
   $$
   \mathbf{b}_1 := \mathbf{b}_1 - \eta \frac{\partial L}{\partial \mathbf{b}_1}
   $$

In [2]:
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
import numpy as np

In [1]:
class NN:
    def __init__(self, layers, learning_rate=0.1, batch_size=32):
        self.layers = layers
        self.learning_rate = learning_rate
        self.batch_size = batch_size

        self.weights = [
            np.random.normal(0.0, pow(layers[i], -0.5), (layers[i + 1], layers[i]))
            for i in range(len(layers) - 1)
        ]
        self.biases = [np.zeros((layers[i + 1], 1)) for i in range(len(layers) - 1)]

    def sigmoid(self, Z):
        return 1 / (1 + np.exp(-Z))

    def sigmoid_prime(self, a):
        return a * (1 - a)

    def forward(self, inputs):
        inputs = np.array(inputs, ndmin=2)
        activations = [inputs]
        for i, (w, b) in enumerate(zip(self.weights, self.biases)):
            z = np.dot(activations[-1], w.T) + b.T
            if i == len(self.weights) - 1:
                # Output layer uses linear activation
                a = z
            else:
                a = self.sigmoid(z)
            activations.append(a)
        return activations

    def cost_function(self, y, t):
        return 0.5 * np.sum((y - t) ** 2)

    def cost_function_prime(self, y, t):
        return y - t

    def backprop(self, X, y, activations):
        batch_size = X.shape[0]
        errors = [self.cost_function_prime(activations[-1], y)]
        for i in reversed(range(len(self.weights))):
            if i == len(self.weights) - 1:
                # Output layer derivative is 1
                delta = errors[-1]
            else:
                delta = errors[-1] * self.sigmoid_prime(activations[i + 1])
            grad_w = np.dot(delta.T, activations[i]) / batch_size
            grad_b = np.mean(delta, axis=0, keepdims=True).T
            errors.append(np.dot(delta, self.weights[i]))
            self.weights[i] -= self.learning_rate * grad_w
            self.biases[i] -= self.learning_rate * grad_b
        errors.reverse()

    def fit(self, inputs_list, targets_list):
        fit_loss = 0
        for i in range(0, len(inputs_list), self.batch_size):
            inputs_batch = np.array(inputs_list[i : i + self.batch_size], ndmin=2)
            targets_batch = np.array(targets_list[i : i + self.batch_size], ndmin=2)

            activations = self.forward(inputs_batch)
            self.backprop(inputs_batch, targets_batch, activations)
            fit_loss += self.cost_function(activations[-1], targets_batch)

        fit_loss /= len(inputs_list)

    def predict(self, inputs_list):
        return self.forward(inputs_list)[-1]

Lets compare its performance with scikit implementation

In [3]:
data = fetch_california_housing()
X = data["data"]
y = data["target"].reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)
y_train = scaler_y.fit_transform(y_train).ravel()  # Flatten for scikit-learn compatibility
y_test = scaler_y.transform(y_test).ravel()

# Train NN 
nn = NN(layers=[8, 20, 10, 1], learning_rate=0.1)
epochs = 100
for epoch in range(epochs):
    nn.fit(X_train, y_train.reshape(-1, 1))  # Reshape to keep consistency

predictions_nn = []
for x in X_test:
    output = nn.predict(x)
    predictions_nn.append(output[0, 0])

predictions_nn = scaler_y.inverse_transform(np.array(predictions_nn).reshape(-1, 1))
y_test_original = scaler_y.inverse_transform(y_test.reshape(-1, 1))

mse_nn = mean_squared_error(y_test_original, predictions_nn)
print(f"Neural Network Mean Squared Error: {mse_nn:.2f}")

# Train scikit-learn Neural Network (MLPRegressor)
mlp_reg = MLPRegressor(hidden_layer_sizes=(20, 10), learning_rate_init=0.1, max_iter=100, random_state=42)
mlp_reg.fit(X_train, y_train)

predictions_mlp = mlp_reg.predict(X_test).reshape(-1, 1)
predictions_mlp = scaler_y.inverse_transform(predictions_mlp)

mse_mlp = mean_squared_error(y_test_original, predictions_mlp)
print(f"scikit-learn Neural Network Mean Squared Error: {mse_mlp:.2f}")

# Train Linear Regression model
linear_reg = LinearRegression()
linear_reg.fit(X_train, y_train)

predictions_lr = linear_reg.predict(X_test)
predictions_lr = scaler_y.inverse_transform(predictions_lr.reshape(-1, 1))

mse_lr = mean_squared_error(y_test_original, predictions_lr)
print(f"Linear Regression Mean Squared Error: {mse_lr:.2f}")

Neural Network Mean Squared Error: 0.32
scikit-learn Neural Network Mean Squared Error: 0.33
Linear Regression Mean Squared Error: 0.56
